In [279]:
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

In [280]:
# IF DATA IS IN YOUR DRIVE
data = pd.read_excel('BLACK_BELT_DATABASE_CASE_FISRT_ANALISYS.xlsx', header=0)

In [281]:
# Função para pré-processar os dados
def preProcessingDataBase(data):

    # PART_NUMBER
    data = data.drop('PART_NUMBER',axis=1)

    # REV
    data = data.drop('REV',axis=1)

    # DESCRIPTION
    data = data.drop('DESCRIPTION',axis=1) #MODIFY TO GROUP BY SIMILARITY

    # RELEASED_DATE_1
    data = data.drop('RELEASED_DATE_1',axis=1)

    # RELEASED_DATE
    data = data.drop('RELEASED_DATE',axis=1)

    # QTN_REV_3D
    data['QTN_REV_3D'] = data['QTN_REV_3D'].dropna()
    data['QTN_REV_3D'] = (data['QTN_REV_3D']-data['QTN_REV_3D'].min())/(data['QTN_REV_3D'].max()-data['QTN_REV_3D'].min())

    # MEAN_SIZE_3D
    data['MEAN_SIZE_3D'] = data['MEAN_SIZE_3D'].dropna()
    data['MEAN_SIZE_3D'] = (data['MEAN_SIZE_3D']-data['MEAN_SIZE_3D'].min())/(data['MEAN_SIZE_3D'].max()-data['MEAN_SIZE_3D'].min())

    # DRAWING_CODE
    data['DRAWING_CODE'] = data['DRAWING_CODE'].dropna()
    data['DRAWING_CODE'] = data['DRAWING_CODE'][data['DRAWING_CODE'] != "EL"]
    data['DRAWING_CODE'] = data['DRAWING_CODE'][data['DRAWING_CODE'] != ""]
    #data = data.join(pd.get_dummies(data.pop('DRAWING_CODE')))

    # ATP
    data['ATP'] = data['ATP'].dropna()
    data['ATP'] = data['ATP'][data['ATP'] != ""]
    #data = data.join(pd.get_dummies(data.pop('ATP')))


    # QTN_REV_2D
    data['QTN_REV_2D'] = data['QTN_REV_2D'].dropna()
    data['QTN_REV_2D'] = (data['QTN_REV_2D']-data['QTN_REV_2D'].min())/(data['QTN_REV_2D'].max()-data['QTN_REV_2D'].min())


    # QTY_ECN_2D
    data['QTY_ECN_2D'] = data['QTY_ECN_2D'].dropna()
    data['QTY_ECN_2D'] = pd.Series(np.searchsorted(['BOM', 'RUIM'], data.QTY_ECN_2D.values), data.index)
    #data['QTY_ECN_2D'] = (data['QTY_ECN_2D']-data['QTY_ECN_2D'].min())/(data['QTY_ECN_2D'].max()-data['QTY_ECN_2D'].min())


    # QTY_ECN_2D_1
    data = data.drop('QTY_ECN_2D_1',axis=1)


    # MEAN_SIZE_2D
    data['MEAN_SIZE_2D'] = data['MEAN_SIZE_2D'].dropna()
    data['MEAN_SIZE_2D'] = (data['MEAN_SIZE_2D']-data['MEAN_SIZE_2D'].min())/(data['MEAN_SIZE_2D'].max()-data['MEAN_SIZE_2D'].min())


    # QTY_SHEETS
    data['QTY_SHEETS'] = data['QTY_SHEETS'].dropna()
    data['QTY_SHEETS'] = (data['QTY_SHEETS']-data['QTY_SHEETS'].min())/(data['QTY_SHEETS'].max()-data['QTY_SHEETS'].min())

    # QTY_DIMENSIONS
    data['QTY_DIMENSIONS'] = data['QTY_DIMENSIONS'].dropna()
    data['QTY_DIMENSIONS'] = (data['QTY_DIMENSIONS']-data['QTY_DIMENSIONS'].min())/(data['QTY_DIMENSIONS'].max()-data['QTY_DIMENSIONS'].min())

    # QTY_VIEWS
    data['QTY_VIEWS'] = data['QTY_VIEWS'].dropna()
    data['QTY_VIEWS'] = (data['QTY_VIEWS']-data['QTY_VIEWS'].min())/(data['QTY_VIEWS'].max()-data['QTY_VIEWS'].min())

    # QTY_PART_LIST
    data['QTY_PART_LIST'] = data['QTY_PART_LIST'].dropna()
    data['QTY_PART_LIST'] = (data['QTY_PART_LIST']-data['QTY_PART_LIST'].min())/(data['QTY_PART_LIST'].max()-data['QTY_PART_LIST'].min())

    # QTY_TEXT_INFORMATION
    data['QTY_TEXT_INFORMATION'] = data['QTY_TEXT_INFORMATION'].dropna()
    data['QTY_TEXT_INFORMATION'] = (data['QTY_TEXT_INFORMATION']-data['QTY_TEXT_INFORMATION'].min())/(data['QTY_TEXT_INFORMATION'].max()-data['QTY_TEXT_INFORMATION'].min())

    # COMPLETED_ON
    data = data.drop('COMPLETED_ON',axis=1)

    # COMPLETED_ON_1
    data = data.drop('COMPLETED_ON_1',axis=1)

    # CREATED_ON
    data = data.drop('CREATED_ON',axis=1)

    # CREATED_ON_1
    data = data.drop('CREATED_ON_1',axis=1)

    # LEAD_TIME_1
    data = data.drop('LEAD_TIME_1',axis=1)

    # LEAD_TIME
    data['LEAD_TIME'] = data['LEAD_TIME'].dropna()
    data['LEAD_TIME'] = data['LEAD_TIME'][data['LEAD_TIME'] > 0]
    data['LEAD_TIME'] = (data['LEAD_TIME']-data['LEAD_TIME'].min())/(data['LEAD_TIME'].max()-data['LEAD_TIME'].min())

    # TRIM_AND_FINISH
    data['TRIM_AND_FINISH'] = data['TRIM_AND_FINISH'].dropna()
    data['TRIM_AND_FINISH'] = (data['TRIM_AND_FINISH']*-1)

    # LEAD_TIME
    data['LEAD_TO_RELEASE'] = data['LEAD_TO_RELEASE'].dropna()
    data['LEAD_TO_RELEASE'] = data['LEAD_TO_RELEASE'][data['LEAD_TO_RELEASE'] > 0]
    data['LEAD_TO_RELEASE'] = (data['LEAD_TO_RELEASE']-data['LEAD_TO_RELEASE'].min())/(data['LEAD_TO_RELEASE'].max()-data['LEAD_TO_RELEASE'].min())

    # DROP ANY ROW NULL
    data = data.dropna()

    return data

In [282]:
# Pré-processing
data = preProcessingDataBase(data)
data, validation_data = train_test_split(data, test_size=0.01)
validation_data, test_data = train_test_split(validation_data, test_size=0.5)

In [283]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('QTY_ECN_2D')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [284]:
feature_columns = []

for header in ['QTN_REV_3D', 'MEAN_SIZE_3D', 'QTN_REV_2D', 'MEAN_SIZE_2D', 'QTY_SHEETS', 'QTY_DIMENSIONS', 'QTY_VIEWS', 'QTY_PART_LIST', 'QTY_TEXT_INFORMATION', 'LEAD_TIME', 'TRIM_AND_FINISH', 'LEAD_TO_RELEASE']:
  feature_columns.append(feature_column.numeric_column(header))


feature_columns.append(feature_column.embedding_column(feature_column.categorical_column_with_vocabulary_list('ATP', data.ATP.unique()), dimension=8))
feature_columns.append(feature_column.embedding_column(feature_column.categorical_column_with_vocabulary_list('DRAWING_CODE', data.DRAWING_CODE.unique()), dimension=8))

In [285]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [286]:
batch_size = 2
train_ds = df_to_dataset(data, batch_size=batch_size)
validation_ds = df_to_dataset(validation_data, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test_data, shuffle=False, batch_size=batch_size)

In [290]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(16, activation='relu'),
  layers.Dropout(.9),
  layers.Dense(8, activation='relu'),
  layers.Dropout(0.75),
  layers.Dense(4, activation='relu'),
  layers.Dropout(.50),
  layers.Dense(1, activation='relu'),
  layers.Dropout(.25),
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(train_ds,
          validation_data=train_ds,
          epochs=10000, 
          callbacks=[tensorboard_callback])



Epoch 1/10000
1337/1337 [==============================] - 37s 22ms/step - loss: 0.7532 - accuracy: 0.7536 - val_loss: 0.6993 - val_accuracy: 0.7745
Epoch 2/10000
1337/1337 [==============================] - 30s 23ms/step - loss: 0.7623 - accuracy: 0.7457 - val_loss: 0.6991 - val_accuracy: 0.7745
Epoch 3/10000
1337/1337 [==============================] - 34s 25ms/step - loss: 0.7414 - accuracy: 0.7521 - val_loss: 0.6990 - val_accuracy: 0.7745
Epoch 4/10000
1337/1337 [==============================] - 29s 22ms/step - loss: 0.7737 - accuracy: 0.7468 - val_loss: 0.6988 - val_accuracy: 0.7745
Epoch 5/10000
1337/1337 [==============================] - 30s 22ms/step - loss: 0.7479 - accuracy: 0.7539 - val_loss: 0.6986 - val_accuracy: 0.7745
Epoch 6/10000
1337/1337 [==============================] - 27s 20ms/step - loss: 0.7544 - accuracy: 0.7532 - val_loss: 0.6984 - val_accuracy: 0.7745
Epoch 7/10000
1337/1337 [==============================] - 28s 21ms/step - loss: 0.7408 - accuracy: 0.7521

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)